In [ ]:
import numpy as np
import pandas as pd



In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)

In [ ]:
movies = movies.merge(credits, on = 'title')

In [ ]:
## now I have to make content based mrs so I will search for cols that are required in this-
## genres(horror)
## id is imp 
## Keywords
## title 
## overview
## cast and crew 

In [ ]:
movies = movies[['id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head(1)

In [ ]:
## now we will make another dataframe and add 3 cols in it - movie id, title and tags- (overview, genres, krywords, cast,crew)
## In cast- we will take only top 3 and in crew only director name.
## 1) We will check null data first 
## 2) We will check for duplicate

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
## now we need to convert this dict into the name but the problem is that we first need to convert 
## list of int/string to list so we will import a function 

In [ ]:
import ast

In [ ]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [ ]:
## now we will apply this to the dataset and improve our generes section
movies['genres'] = movies['genres'].apply(convert)
movies.head()


In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

In [ ]:
def convert3(text):
    L = []
    counter=0
    for i in ast.literal_eval(text):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
        
    return L 

In [ ]:
movies['cast'].apply(convert3)

In [ ]:
movies.head(1)
### only crew left now in which we need director 

In [ ]:
movies['crew'][0]
## in this we need the job title to be director and its name in dict 

In [ ]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.sample(5)

In [ ]:
movies['overview'][1]

In [ ]:
movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
import ast

In [ ]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)


In [ ]:
movies.head()

In [ ]:
## now the issue is that while creating tags eg- Johny Depp are 2 diff words so we need to create 1 tage we will use lambda func
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [ ]:
movies.head()

In [ ]:
## now we will create one tags col in which we will conacatenate all these 5 cols 
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head(5)## new col created tags under which overview, genres,etc all added 

In [ ]:
## now we need to remove cols rest one 
new_df=movies[['id','title','tags']]

In [ ]:
new_df.head(5)

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head(5)

In [ ]:
## now we have to start with text vectorization and before that if we understand our problem statement, 
## it would take a movie as an input and return the top 5 movies same as that. 
# 1 method is to compare the no. of similar words in both tags and calculate top 5 with max similar words
## but rather than this we can do vectorization-convert string to vector
## we will assign every vector a movie and if we select a movie, we will pick the five closest vectors 
## text->vectors(text vectorization) *bag of words, ifidf, word2vec
## we will concatenate all tags suppose / tag 100 words total =500000
## now out of these 500000 words i will select the top 5000 most repetitive words
## we will create a 2D matrix every row is a vector and m1- how many times the most occured 5000th word (5000,5000)
## check best results kisme mil rhe hai 2000 5000 10,000 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector = cv.fit_transform(new_df['tags']).toarray()


In [ ]:
vector.shape ## movies- 4806 and 5000 top words 

In [ ]:
vector

In [ ]:
vector[0]

In [ ]:
len(cv.get_feature_names())

In [ ]:
## now we have words where action actions activity activities etc repetitive words so we will use nltk lib and 
## import stem which converts similar words to 1 word 

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    string=" ".join(y)
    

In [ ]:
##new_df['tags']=new_df['tags'].apply(stem) ## 1:16:51 this was not working 

In [62]:
## now we will have to calculate the distance of every movie from the particular movie selected
## more the distance, less related the movies are(as the movies now are vectors)
## we will not calculate the euclidean distance, we will calculate the cosine distance 
## as the dimensionality increases, the measure of euclidean fails 
## we will use cosine similarity if 0- less similar if 1 then more similar

In [63]:
from sklearn.metrics.pairwise import cosine_similarity

In [64]:
similarity = cosine_similarity(vector)

In [65]:
similarity.shape ## it will be 4806x4806 that is it will have these dimensions 

(4806, 4806)

In [66]:
similarity[0] ## here we have the similarity status of the first movie with every movie. 

array([1.        , 0.08964215, 0.06071767, ..., 0.02519763, 0.0277885 ,
       0.        ])

In [77]:
## now we will create a recommend function which will give top 5 similar movies. 
## enumerate list func- it is used because if we sort the array without that, then we will losse our index values
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [78]:
distances = sorted(list(enumerate(similarity[0])),reverse=True,key = lambda x: x[1])
print(distances) ## in this way it will not be of any trouble
## the lambda function tells that we have to sort from second col 

[(0, 1.0), (539, 0.26089696604360174), (1194, 0.2581988897471611), (260, 0.25110592822973776), (1216, 0.24944382578492943), (507, 0.24846467329894412), (1444, 0.24397501823713333), (1920, 0.243599382882345), (582, 0.24147264420814754), (3730, 0.23904572186687872), (74, 0.22677868380553634), (3608, 0.2238868314198225), (322, 0.22230800575069137), (83, 0.22190115272469205), (4192, 0.22131333406899523), (2333, 0.2173253797873328), (1204, 0.21653278478430665), (47, 0.2164218276749025), (61, 0.2162249910469341), (495, 0.2162249910469341), (2409, 0.2138089935299395), (1537, 0.2125118592516207), (942, 0.21251185925162067), (972, 0.21251185925162067), (466, 0.2114722130550724), (1329, 0.21147221305507238), (2204, 0.2059714602177749), (151, 0.20498001542269692), (1201, 0.20283702113484398), (973, 0.2), (4048, 0.1980534816610477), (3162, 0.19518001458970666), (4405, 0.19518001458970666), (1275, 0.19518001458970663), (2786, 0.19518001458970663), (2971, 0.19477964490741226), (3538, 0.1945195050318

In [80]:
recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman
